In [ ]:
# Download the datatset from Kaggle.
!kaggle datasets download -p datasets/face-recognition -d sbaghbidi/human-faces-object-detection --unzip
# Split CSV into training, validation and testing data
!python csv_split_dataset.py --csv datasets/face-recognition/faces.csv --images datasets/face-recognition/images
# Convert CSV to TFRecord
!python csv_to_tfrecord.py --input_path datasets/face-recognition/train/faces.csv --output_path datasets/face-recognition/train/faces.record
!python csv_to_tfrecord.py --input_path datasets/face-recognition/val/faces.csv --output_path datasets/face-recognition/val/faces.record
!python csv_to_tfrecord.py --input_path datasets/face-recognition/test/faces.csv --output_path datasets/face-recognition/test/faces.record
# Create a label map for this dataset
!echo "item { id: 1 name: 'face' }" > datasets/face-recognition/label_map.pbtxt

# Download and install TensorFlow Object Detection API
!brew install protobuf
!git clone --depth 1 https://github.com/tensorflow/models
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
%cd ../../

# Set up directory structure
!mkdir pre-trained-models
!mkdir training

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Import the TensorFlow Object Detection API visualisation tools
from object_detection.utils import config_util, label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.protos import pipeline_pb2

# Display plots in Jupyter
%matplotlib inline

In [ ]:
MODEL = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
model_url = f'http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL}.tar.gz'
MODEL_DIR = str(tf.keras.utils.get_file(fname=MODEL, origin=model_url, cache_dir='pre-trained-models/', untar=True))

configs = config_util.get_configs_from_pipeline_file(f'{MODEL_DIR}/pipeline.config')

configs['model'].ssd.num_classes = 1
configs['train_config'].batch_size = 2
configs['train_config'].fine_tune_checkpoint = f'{MODEL_DIR}/checkpoint/ckpt-0'
configs['train_config'].num_steps = 10000
configs['train_config'].fine_tune_checkpoint_type= 'detection'
configs['train_input_config'].label_map_path = 'datasets/face-recognition/label_map.pbtxt'
configs['train_input_config'].tf_record_input_reader.input_path[0] = 'datasets/face-recognition/train/faces.record'
configs['eval_input_config'].label_map_path = 'datasets/face-recognition/label_map.pbtxt'
configs['eval_input_config'].tf_record_input_reader.input_path[0] = 'datasets/face-recognition/val/faces.record'

pipeline_proto = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_proto, MODEL_DIR)

In [ ]:
# tensorboard --logdir training/ssd_mobilenet_v2_320x320_coco17_tpu-8/train/
!python models/research/object_detection/model_main_tf2.py --model_dir=training/ssd_mobilenet_v2_320x320_coco17_tpu-8 --pipeline_config_path=pre-trained-models/datasets/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config

In [ ]:
# Evaluation
!python models/research/object_detection/model_main_tf2.py --model_dir=training/ssd_mobilenet_v2_320x320_coco17_tpu-8 --pipeline_config_path=pre-trained-models/datasets/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config --checkpoint_dir=training/ssd_mobilenet_v2_320x320_coco17_tpu-8

In [ ]:
def load_image(path):
    print(f'Loading image - {path}')
    if path.startswith('http'):
        return np.array(Image.open(tf.keras.utils.get_file(origin=path)))
    return np.array(Image.open(path)) 

def load_label_map(path):
    print(f'Loading label map - {path}')
    return label_map_util.create_category_index_from_labelmap(path)

def load_model(checkpoint_dir):
    latest_ckpt = tf.train.latest_checkpoint(checkpoint_dir)
    print(f'Loading checkpoint - {latest_ckpt}')
    # Load pipeline config
    configs = config_util.get_configs_from_pipeline_file(f'{MODEL_DIR}/pipeline.config')
    model = model_builder.build(model_config=configs['model'], is_training=False)

    # Restore checkpoint
    ckpt = tf.compat.v2.train.Checkpoint(model=model)
    ckpt.restore(latest_ckpt).expect_partial()

    return model

In [ ]:
# Test image included in TensorFlow Object Detection API.
IMAGE_PATH = 'https://upload.wikimedia.org/wikipedia/commons/a/a0/Pierre-Person.jpg'
LABEL_MAP_PATH = 'datasets/face-recognition/label_map.pbtxt'
CHECKPOINT_PATH = 'training/ssd_mobilenet_v2_320x320_coco17_tpu-8/' # Path to last checkpoint created while training

image = load_image(IMAGE_PATH)
labels = load_label_map(LABEL_MAP_PATH)
model = load_model(CHECKPOINT_PATH)